In [0]:
#Get data from google drive
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [5]:
#Import Libraries
import pandas as pd
from gensim.models.doc2vec import Doc2Vec
from nltk import word_tokenize
from joblib import parallel_backend, Parallel, delayed
from tqdm import *
import numpy as np

import nltk
nltk.download("punkt")

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [0]:
#Import article df
articles = pd.read_csv("/content/drive/My Drive/article_df.csv")
articles.head()

,paper_id,title,abstract,text,type
0,b2f67d533f2749807f2537f3775b39da3b186051,Caring for persons in detention suffering with...,NaN,Caring for persons in detention suffering with...,non-commercial
1,ad98979eada6e333a276d39efdce21779d538625,Xanthine-based acyclic nucleoside phosphonates...,While noncanonic xanthine nucleotides XMP/dXMP...,Xanthine-based acyclic nucleoside phosphonates...,non-commercial
2,464f7d3a460eb51dbc25bd12639b22079a73f85a,Viral lncRNA: A regulatory molecule for contro...,Long non-coding RNAs (lncRNAs) are found not o...,Viral lncRNA: A regulatory molecule for contro...,non-commercial
3,c436139975d97ef929b5d8452595de40bda0c11c,"Phone: (1) 301-451-9881, jbeigel@niaid.nih.gov",on behalf of the IRC002 Study Team Summary Bac...,"Phone: (1) 301-451-9881, jbeigel@niaid.nih.gov...",non-commercial
4,634128ea7d7736750e1c3cd0a48bb37843d06dac,A Strategy To Estimate Unknown Viral Diversity...,The majority of emerging zoonoses originate in...,A Strategy To Estimate Unknown Viral Diversity...,non-commercial


In [31]:
#Import doc2vec
model = Doc2Vec.load("/content/drive/My Drive/doc2vec.bin")

/usr/local/lib/python3.6/dist-packages/smart_open/smart_open_lib.py:410: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL
/usr/local/lib/python3.6/dist-packages/gensim/models/doc2vec.py:566: UserWarning: The parameter `iter` is deprecated, will be removed in 4.0.0, use `epochs` instead.
  warnings.warn("The parameter `iter` is deprecated, will be removed in 4.0.0, use `epochs` instead.")
/usr/local/lib/python3.6/dist-packages/gensim/models/doc2vec.py:570: UserWarning: The parameter `size` is deprecated, will be removed in 4.0.0, use `vector_size` instead.
  warnings.warn("The parameter `size` is deprecated, will be removed in 4.0.0, use `vector_size` instead.")


In [0]:
#Function: calculate doc2vec
def calculate_doc2vec(i, df=articles, model=model, epochs=100):

  #tokenize article text
  title_abstract = df.title[i] + " " + df.abstract[i]
  tokenized = word_tokenize(title_abstract.lower())
  doc2vec = model.infer_vector(tokenized, epochs=epochs).tolist()

  articles.loc[i, "doc2vec_title_abstract"] = doc2vec

In [74]:
#Add column doc2vec
articles = articles.dropna().reset_index(drop=True)
articles["doc2vec"] = [[]] * articles.shape[0]
articles.head()

,paper_id,title,abstract,text,type,doc2vec,doc2vec_title_abstract
0,ad98979eada6e333a276d39efdce21779d538625,Xanthine-based acyclic nucleoside phosphonates...,While noncanonic xanthine nucleotides XMP/dXMP...,Xanthine-based acyclic nucleoside phosphonates...,non-commercial,"[-0.21714907884597778, -0.3641749918460846, -0...",[]
1,464f7d3a460eb51dbc25bd12639b22079a73f85a,Viral lncRNA: A regulatory molecule for contro...,Long non-coding RNAs (lncRNAs) are found not o...,Viral lncRNA: A regulatory molecule for contro...,non-commercial,"[-0.19442467391490936, -0.0742512047290802, -0...",[]
2,c436139975d97ef929b5d8452595de40bda0c11c,"Phone: (1) 301-451-9881, jbeigel@niaid.nih.gov",on behalf of the IRC002 Study Team Summary Bac...,"Phone: (1) 301-451-9881, jbeigel@niaid.nih.gov...",non-commercial,"[0.14343851804733276, 0.1714065968990326, -0.3...",[]
3,634128ea7d7736750e1c3cd0a48bb37843d06dac,A Strategy To Estimate Unknown Viral Diversity...,The majority of emerging zoonoses originate in...,A Strategy To Estimate Unknown Viral Diversity...,non-commercial,"[-0.522060751914978, 0.021631771698594093, -0....",[]
4,fef0bb9eaac69559d0ff2f92ff83e0affd4435f0,Pathogenic Influenza Viruses and Coronaviruses...,The broad range and diversity of interferon-st...,Pathogenic Influenza Viruses and Coronaviruses...,non-commercial,"[-0.029732080176472664, 0.6708524227142334, -0...",[]


In [77]:
#Apply doc2vec on all the articles
with parallel_backend("threading", n_jobs=40):
  Parallel()(delayed(calculate_doc2vec)(i) for i in tqdm(range(articles.shape[0])))

100%|██████████| 23857/23857 [22:35<00:00, 17.60it/s]


In [0]:
#Save the new DF
articles.to_pickle("/content/drive/My Drive/articles_new2.pkl")

In [43]:
#Read new articles
articles = pd.read_pickle("/content/drive/My Drive/articles_new.pkl")
articles = articles.dropna().reset_index(drop=True)
articles.head()

,paper_id,title,abstract,text,type,doc2vec
0,ad98979eada6e333a276d39efdce21779d538625,Xanthine-based acyclic nucleoside phosphonates...,While noncanonic xanthine nucleotides XMP/dXMP...,Xanthine-based acyclic nucleoside phosphonates...,non-commercial,"[-0.21714907884597778, -0.3641749918460846, -0..."
1,464f7d3a460eb51dbc25bd12639b22079a73f85a,Viral lncRNA: A regulatory molecule for contro...,Long non-coding RNAs (lncRNAs) are found not o...,Viral lncRNA: A regulatory molecule for contro...,non-commercial,"[-0.19442467391490936, -0.0742512047290802, -0..."
2,c436139975d97ef929b5d8452595de40bda0c11c,"Phone: (1) 301-451-9881, jbeigel@niaid.nih.gov",on behalf of the IRC002 Study Team Summary Bac...,"Phone: (1) 301-451-9881, jbeigel@niaid.nih.gov...",non-commercial,"[0.14343851804733276, 0.1714065968990326, -0.3..."
3,634128ea7d7736750e1c3cd0a48bb37843d06dac,A Strategy To Estimate Unknown Viral Diversity...,The majority of emerging zoonoses originate in...,A Strategy To Estimate Unknown Viral Diversity...,non-commercial,"[-0.522060751914978, 0.021631771698594093, -0...."
4,fef0bb9eaac69559d0ff2f92ff83e0affd4435f0,Pathogenic Influenza Viruses and Coronaviruses...,The broad range and diversity of interferon-st...,Pathogenic Influenza Viruses and Coronaviruses...,non-commercial,"[-0.029732080176472664, 0.6708524227142334, -0..."


In [0]:
#Import doc2vec
model = Doc2Vec.load("/content/drive/My Drive/doc2vec.bin")

In [0]:
#Function: Calculate similarity
def calculate_similarity(query_text, df, top_n=10):

  #Build doc2vec for the query text
  query_tokenized = word_tokenize(query_text.lower())
  query_doc2vec = model.infer_vector(query_tokenized, epochs=100)

  #Extract loaded doc2vec
  loaded_doc2vec = np.array(df.doc2vec_title_abstract.tolist())

  #calculate similarity
  similarity_l2 = np.linalg.norm(loaded_doc2vec - query_doc2vec, axis = 1)
  top_results = np.argsort(similarity_l2)[:top_n].tolist()
  print(top_results)


  #Print Results
  for i in top_results:
    print("+"*130)
    print("Title:\n")
    print(df.title[i])
    print("\nAbstract:\n")
    print(df.abstract[i])